In [2]:
import pandas as pd
import numpy as np
from pulp import *
import re

In [3]:
df=pd.read_csv("FPL_2018_19_Wk0.csv")
df.head()

,Name,Team,Position,Cost,Creativity,Influence,Threat,ICT,Goals_conceded,Goals_scored,...,Own_goals,Penalties_missed,Penalties_saved,Saves,Yellow_cards,Red_cards,TSB,Minutes,Bonus,Points
0,Adam Smith,BOU,DEF,45,345.5,455.0,144.0,94.5,38,1,...,0,0,0,0,6,0,0.3,2067,3,56
1,Adrian,WHU,GKP,45,0.0,470.4,0.0,47.0,29,0,...,0,0,0,69,2,0,0.6,1710,5,72
2,Aguero,MCI,FWD,110,570.8,966.4,1484.0,302.5,12,21,...,0,0,0,0,2,0,12.6,1960,22,169
3,Ake,BOU,DEF,50,115.1,932.4,287.0,133.5,59,2,...,0,0,0,0,5,0,5.7,3352,8,102
4,Albrighton,LEI,MID,55,718.3,580.0,300.0,160.2,42,2,...,0,0,0,0,5,1,1.1,2533,12,107


In [64]:
df[['Name','Assists','Yellow_cards','Goals_scored','Minutes']].head()

,Name,Assists,Yellow_cards,Goals_scored,Minutes
0,Adam Smith,3,6,1,2067
1,Adrian,0,2,0,1710
2,Aguero,6,2,21,1960
3,Ake,3,5,2,3352
4,Albrighton,8,5,2,2533


In [71]:
df.columns
select=['Name', 'Team', 'Position', 'Cost','Assists', 'Yellow_cards','Minutes', 'Points']
df[select].head(10).to_clipboard(index=False, sep=',')

In [4]:
cov_df=df.iloc[:,5:21]

In [5]:
cov_df.corr()

,Influence,Threat,ICT,Goals_conceded,Goals_scored,Assists,Own_goals,Penalties_missed,Penalties_saved,Saves,Yellow_cards,Red_cards,TSB,Minutes,Bonus,Points
Influence,1.000000,0.615033,0.853733,0.819560,0.562604,0.582243,0.203212,0.199690,0.219277,0.275144,0.591854,0.148540,0.530426,0.932931,0.801257,0.935360
Threat,0.615033,1.000000,0.891189,0.391405,0.921886,0.719974,-0.016251,0.388562,-0.108708,-0.146953,0.340849,0.077265,0.481659,0.500678,0.683407,0.731102
ICT,0.853733,0.891189,1.000000,0.639134,0.798181,0.818411,0.062853,0.303271,-0.002292,-0.011208,0.499486,0.133306,0.537459,0.767663,0.804940,0.906861
Goals_conceded,0.819560,0.391405,0.639134,1.000000,0.280112,0.381055,0.189435,0.146886,0.221756,0.283956,0.641296,0.168479,0.253414,0.918785,0.529923,0.727945
Goals_scored,0.562604,0.921886,0.798181,0.280112,1.000000,0.606734,-0.037963,0.451427,-0.079517,-0.107519,0.243355,0.027032,0.513850,0.388958,0.731513,0.682152
Assists,0.582243,0.719974,0.818411,0.381055,0.606734,1.000000,-0.020390,0.182238,-0.091161,-0.126850,0.295796,0.124612,0.425800,0.493862,0.602426,0.691898
Own_goals,0.203212,-0.016251,0.062853,0.189435,-0.037963,-0.020390,1.000000,-0.032142,0.087850,0.056229,0.146970,0.004709,0.015544,0.203181,0.127151,0.127558
Penalties_missed,0.199690,0.388562,0.303271,0.146886,0.451427,0.182238,-0.032142,1.000000,-0.027435,-0.037096,0.163158,0.055211,0.183236,0.152014,0.310170,0.242251
Penalties_saved,0.219277,-0.108708,-0.002292,0.221756,-0.079517,-0.091161,0.087850,-0.027435,1.000000,0.740837,-0.068201,-0.043405,0.147180,0.225176,0.164908,0.205095
Saves,0.275144,-0.146953,-0.011208,0.283956,-0.107519,-0.126850,0.056229,-0.037096,0.740837,1.000000,-0.088331,-0.058691,0.192678,0.291857,0.182247,0.254424


In [6]:
df.dtypes

Name                 object
Team                 object
Position             object
Cost                  int64
Creativity          float64
Influence           float64
Threat              float64
ICT                 float64
Goals_conceded        int64
Goals_scored          int64
Assists               int64
Own_goals             int64
Penalties_missed      int64
Penalties_saved       int64
Saves                 int64
Yellow_cards          int64
Red_cards             int64
TSB                 float64
Minutes               int64
Bonus                 int64
Points                int64
dtype: object

In [7]:
# create variable x1,x2,x3....
def player_variable(df):
    player_variable = []
    
    for rownum, row in df.iterrows():
        variable = str('x' + str(rownum))
        variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer')
        player_variable.append(variable)
                                  
    return player_variable

In [8]:
def objective_func(df,player_list,lp_prob):
    tot_pt=""
    for n,row in df.iterrows():
        for i,player in enumerate(player_list):
            if i==n:
                formula=row['Points']*player
                tot_pt+=formula
    lp_prob+=tot_pt
    return lp_prob
    
    

In [9]:
def cash_constraint(df,player_list,lp_prob,available_cash):
    cost=""
    formula=""
    for n,row in df.iterrows():
        for i,player in enumerate(player_list):
            if i==n:
                formula=row['Cost']*player
                cost+=formula
    lp_prob+=(cost<=available_cash),"Cash_constraint"
    return lp_prob
    

In [10]:
def positions(df,player_list,lp_prob,available_player,position):
    tot_player=""
    formula=""
    for n,row in df.iterrows():
        for i,player in enumerate(player_list):
            if i==n:
                if row['Position']==position:
                    formula=1*player
                    tot_player+=formula
    lp_prob+=(tot_player==available_player),position
    return lp_prob

In [11]:
def assists(df,player_list,lp_prob,min_assist):
    tot_assists=""
    formula=""
    for n,row in df.iterrows():
        for i,player in enumerate(player_list):
            if i==n:
                formula=row["Assists"]*player
                tot_assists+=formula
    lp_prob+=(tot_assists>=min_assist),"Assists"
    return lp_prob

In [12]:
def yellow_cards(df,player_list,lp_prob,max_yellow):
    tot_yellow=""
    formula=""
    for n,row in df.iterrows():
        for i,player in enumerate(player_list):
            if i==n:
                formula=row["Yellow_cards"]*(1)*player
                tot_yellow+=formula
    lp_prob+=(tot_yellow<=max_yellow),"Yellow_cards"
    return lp_prob

In [13]:
def goals_scored(df,player_list,lp_prob,min_goal):
    tot_goals=""
    formula=""
    for n,row in df.iterrows():
        for i,player in enumerate(player_list):
            if i==n:
                formula=row["Goals_scored"]*player
                tot_goals+=formula
    lp_prob+=(tot_goals>=min_goal),"Goals_scored"
    return lp_prob

In [14]:
def minutes(df,player_list,lp_prob,min_minute):
    tot_minutes=""
    formula=""
    for n,row in df.iterrows():
        for i,player in enumerate(player_list):
            if i==n:
                formula=row["Minutes"]*player
                tot_minutes+=formula
    lp_prob+=(tot_minutes>=min_minute),"Minutes Played"
    return lp_prob

In [38]:
#remove assist and goals
def find_prob(df,cash,gkp,defd,mid,fwd,yellow,minute):
    lp_prob=[]
    lp_prob = pulp.LpProblem('FantasyTeam', pulp.LpMaximize)
    player_list = player_variable(df)
    
    lp_prob = objective_func(df,player_list,lp_prob)
    lp_prob = cash_constraint(df,player_list,lp_prob,cash)
    lp_prob = positions(df,player_list,lp_prob,gkp,"GKP")
    lp_prob = positions(df,player_list,lp_prob,defd,"DEF")
    lp_prob = positions(df,player_list,lp_prob,mid,"MID")
    lp_prob = positions(df,player_list,lp_prob,fwd,"FWD")
    lp_prob = yellow_cards(df,player_list,lp_prob,yellow)
    lp_prob = minutes(df,player_list,lp_prob,minute)
    
    return lp_prob

In [54]:
def find_prob(df,cash,gkp,defd,mid,fwd,assist,yellow,goals,minute):
    lp_prob=[]
    lp_prob = pulp.LpProblem('FantasyTeam', pulp.LpMaximize)
    player_list = player_variable(df)
    
    lp_prob = objective_func(df,player_list,lp_prob)
    lp_prob = cash_constraint(df,player_list,lp_prob,cash)
    lp_prob = positions(df,player_list,lp_prob,gkp,"GKP")
    lp_prob = positions(df,player_list,lp_prob,defd,"DEF")
    lp_prob = positions(df,player_list,lp_prob,mid,"MID")
    lp_prob = positions(df,player_list,lp_prob,fwd,"FWD")
    #lp_prob = goalkeeper(df,player_list,lp_prob,gkp)
    #lp_prob = defender(df,player_list,lp_prob,defd)
    #lp_prob = midfielder(df,player_list,lp_prob,mid)
    #lp_prob = forward(df,player_list,lp_prob,fwd)
    lp_prob = assists(df,player_list,lp_prob,assist)
    lp_prob = yellow_cards(df,player_list,lp_prob,yellow)
    lp_prob = goals_scored(df,player_list,lp_prob,goals)
    lp_prob = minutes(df,player_list,lp_prob,minute)
    
    return lp_prob

In [16]:
def optimization(df, lp_prob):
    lp_prob.writeLP('FantasyTeam.lp')
    
    optimization_result = lp_prob.solve()
    assert optimization_result == pulp.LpStatusOptimal
    #return optimization_result


In [17]:
def decision(df,lp_prob):
    # append the variable x1,x2... and decison value 1,0 into dataframe
    # the index of vals in ascending order, but columns of variables are not. 
    name,val,vals=[],[],[]
    for v in lp_prob.variables():
        name.append(v.name)
        val.append(v.varValue)
    vals=pd.DataFrame({'variable': name,'value': val}) 
    
    # sort the value of column's variable in ascending order
    for n, row in vals.iterrows():
        value = re.findall(r'(\d+)', row['variable'])
        vals.loc[n, 'variable'] = int(value[0])

    df_vals = vals.sort_index(by='variable')
    
    for n,row in df.iterrows():
        for vals_n,vals_row in df_vals.iterrows():
            if n==vals_row["variable"]:
                df.loc[n,'decision']=vals_row['value']
    
    return df

In [43]:
#try removing assist and goals
def diff_formation(df,cash,gkp,defd,mid,fwd,yellow,minute):
    lp_prob=[]
    lp_prob.clear()
    lp_prob=find_prob(df,cash,gkp,defd,mid,fwd,yellow,minute)
    optimization(df,lp_prob)
    final=decision(df,lp_prob)
    print("Total Cost: ",final[final['decision']==1.0].Cost.sum(), \
          "Total Points: ",final[final['decision']==1.0].Points.sum(), \
         "Total Yellow Cards: ",final[final['decision']==1.0].Yellow_cards.sum(), \
          "Total Minutes :",final[final['decision']==1.0].Minutes.sum())
    return(final[final['decision']==1.0],lp_prob)

In [58]:
def diff_formation(df,cash,gkp,defd,mid,fwd,assist,yellow,goals,minute):
    lp_prob=[]
    lp_prob.clear()
    lp_prob=find_prob(df,cash,gkp,defd,mid,fwd,assist,yellow,goals,minute)
    optimization(df,lp_prob)
    final=decision(df,lp_prob)
    print("Total Cost: ",final[final['decision']==1.0].Cost.sum(),"Total Points: ", \
          final[final['decision']==1.0].Points.sum(), \
          "Total Goals: ",final[final['decision']==1.0].Goals_scored.sum(), \
          "Total Assists: ",final[final['decision']==1.0].Assists.sum(), \
         "Total Yellow Cards: ",final[final['decision']==1.0].Yellow_cards.sum(), \
          "Total Minutes :",final[final['decision']==1.0].Minutes.sum())
    return(final[final['decision']==1.0],lp_prob)

In [59]:
final,lp_prob=diff_formation(df,1200,2,5,5,3,90,20,150,44100)
# final,lp_prob=diff_formation(df,1200,2,5,5,3,20,44100) #remove assist and goal
# final,lp_prob=diff_formation(df,1200,2,5,5,3,20,1315) #remove assist and goal

C:\Users\mycp2fky\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  from ipykernel import kernelapp as app


Total Cost:  1190 Total Points:  2577 Total Goals:  152 Total Assists:  92 Total Yellow Cards:  20 Total Minutes : 44186


In [68]:
final[select].to_clipboard(index=False, sep=',')

In [66]:
# final[final['decision']==1.0]
final[select]
# df[select].head(20)

,Name,Team,Position,Cost,Assists,Yellow_cards,Minutes,Points,decision
22,Azpilicueta,CHE,DEF,65,6,1,3330,175,1.0
45,Bertrand,SOU,DEF,50,5,2,3135,103,1.0
80,Christensen,CHE,DEF,55,0,0,2067,79,1.0
97,Daniels,BOU,DEF,45,3,0,3001,94,1.0
138,Eriksen,TOT,MID,95,10,0,3218,199,1.0
141,Fabianski,WHU,GKP,45,0,0,3420,157,1.0
147,Firmino,LIV,FWD,95,8,1,2760,181,1.0
221,Kane,TOT,FWD,125,2,5,3074,217,1.0
273,Mahrez,MCI,MID,90,13,2,2948,195,1.0
368,Ryan,BHA,GKP,45,0,0,3420,146,1.0


In [30]:
final[['Cost','Points']][final['decision']==1.0].sum()

Cost      1190
Points    2577
dtype: int64

# Testing code

In [41]:
final[final['decision']==1.0].Saves.sum()

283

In [8]:
tot_gk=""
formula=""
lp_prob=[]
player_list = player_variable(df)
position="GKP"
for n,row in df.iterrows():
    if row['Saves']>40 and row['Position']==position:
        print(row['Saves'],type(row['Saves']))
    #for i,player in enumerate(player_list):
        #if i==n:
            #if row['Position']=="GKP" and int(row['Saves'])>int(40):
                #formula=1*player
                #tot_gk+=formula
#lp_prob+=(tot_gk==2),"Goalkeeper"
#print(lp_prob)


69 <class 'int'>
111 <class 'int'>
90 <class 'int'>
77 <class 'int'>
115 <class 'int'>
58 <class 'int'>
44 <class 'int'>
137 <class 'int'>
69 <class 'int'>
99 <class 'int'>
61 <class 'int'>
91 <class 'int'>
87 <class 'int'>
103 <class 'int'>
42 <class 'int'>
121 <class 'int'>
113 <class 'int'>
124 <class 'int'>
93 <class 'int'>


In [68]:
lp_prob=[]
tot_goals=""
formula=""
player_list=player_variable(df)
for n,row in df.iterrows():
    for i,player in enumerate(player_list):
        if i==n:
            formula=row["Goals_scored"]*(-1)*player
            tot_goals+=formula
lp_prob+=(tot_goals<=250),"Goals_scored"



    

In [22]:

print(lp_prob)

FantasyTeam:
MAXIMIZE
56*x0 + 72*x1 + 13*x10 + 33*x100 + 27*x101 + 32*x102 + 169*x103 + 143*x104 + 66*x105 + 71*x106 + 209*x107 + 172*x108 + 73*x109 + 1*x11 + 60*x110 + 62*x111 + 55*x112 + 52*x113 + 94*x114 + 25*x115 + 21*x116 + 70*x118 + 5*x119 + 85*x120 + 136*x123 + 24*x125 + 53*x126 + 100*x127 + 52*x128 + 97*x129 + 62*x13 + 158*x132 + 57*x133 + 22*x134 + 14*x136 + 199*x138 + 78*x140 + 157*x141 + 47*x144 + 29*x145 + 120*x146 + 181*x147 + 8*x148 + 144*x15 + 73*x150 + 123*x151 + 56*x152 + 97*x153 + 57*x156 + 89*x157 + 66*x158 + 94*x159 + 27*x16 + 77*x161 + 64*x162 + 86*x163 + 59*x164 + 80*x165 + 79*x166 + 164*x167 + 117*x169 + 78*x17 + 87*x171 + 82*x174 + 51*x175 + 9*x177 + 41*x178 + 173*x179 + 87*x18 + 8*x180 + 43*x181 + 75*x182 + 85*x183 + 99*x184 + 44*x185 + 74*x186 + 53*x187 + 27*x189 + 77*x19 + 72*x190 + 37*x191 + 2*x192 + 53*x193 + 3*x194 + 1*x195 + 39*x196 + 85*x197 + 58*x198 + 54*x199 + 169*x2 + 72*x20 + 82*x200 + 18*x201 + 89*x202 + 101*x203 + 60*x205 + 4*x206 + 26*x207 + 59*x

In [41]:
lp_prob=find_prob(df,1000,2,5,5,3,25,30,100)
optimization(df,lp_prob)

In [73]:
final=decision(df,lp_prob)
#print(final[final['decision']==1.0].Cost.sum(),final[final['decision']==1.0].Points.sum())
print(final[final['decision']==1.0].Cost.sum(),final[final['decision']==1.0].Points.sum())

AttributeError: 'list' object has no attribute 'variables'

In [138]:
for n, row in vals.iterrows():
        value = re.findall(r'(\d+)', row['variable'])
        vals.loc[n, 'variable'] = int(value[0])

df_vals = vals.sort_index(by='variable')

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """


In [139]:
for n,row in df.iterrows():
    for vals_n,vals_row in df_vals.iterrows():
        if n==vals_row["variable"]:
            df.loc[n,'decision']=vals_row['value']

In [109]:
df[df['decision']==1.0].Cost.sum()

440

In [110]:
re.findall(r'(\d+)','x10')
#for vals_n,row in df_vals.iterrows():
 #   print(vals_n)

['10']

In [71]:
for v in lp_prob.variables():
    print(v.name,"=",v.varValue)

x0 = 0.0
x1 = 0.0
x10 = 0.0
x100 = 0.0
x101 = 0.0
x102 = 0.0
x103 = 0.0
x104 = 0.0
x105 = 0.0
x106 = 0.0
x107 = 0.0
x108 = 0.0
x109 = 0.0
x11 = 0.0
x110 = 0.0
x111 = 0.0
x112 = 0.0
x113 = 0.0
x114 = 0.0
x115 = 0.0
x116 = 0.0
x117 = 0.0
x118 = 0.0
x119 = 0.0
x12 = 0.0
x120 = 0.0
x121 = 0.0
x122 = 0.0
x123 = 0.0
x124 = 0.0
x125 = 0.0
x126 = 0.0
x127 = 0.0
x128 = 0.0
x129 = 0.0
x13 = 0.0
x130 = 0.0
x131 = 0.0
x132 = 0.0
x133 = 0.0
x134 = 0.0
x135 = 0.0
x136 = 0.0
x137 = 0.0
x138 = 0.0
x139 = 0.0
x14 = 0.0
x140 = 0.0
x141 = 0.0
x142 = 0.0
x143 = 0.0
x144 = 0.0
x145 = 0.0
x146 = 0.0
x147 = 0.0
x148 = 0.0
x149 = 0.0
x15 = 0.0
x150 = 0.0
x151 = 0.0
x152 = 0.0
x153 = 0.0
x154 = 0.0
x155 = 0.0
x156 = 0.0
x157 = 0.0
x158 = 0.0
x159 = 0.0
x16 = 0.0
x160 = 0.0
x161 = 0.0
x162 = 0.0
x163 = 0.0
x164 = 0.0
x165 = 0.0
x166 = 0.0
x167 = 0.0
x168 = 0.0
x169 = 0.0
x17 = 0.0
x170 = 0.0
x171 = 0.0
x172 = 0.0
x173 = 0.0
x174 = 0.0
x175 = 0.0
x176 = 0.0
x177 = 0.0
x178 = 0.0
x179 = 0.0
x18 = 0.0
x180 = 0.0
x